# Dymanics in the three-body decay

In [ ]:
using InstantiateFromURL
github_project("mmikhasenko/Introduction2AmplitudeAnalysis")

1. Scattering amplitude: d.o.f.
2. $|M|^2 = 1$ $\Rightarrow$ flat Dalitz
3. Subchannel resonances: bands on the Dalitz plot
4. Projections and reflections
5. Intensity along the line band

In [ ]:
A(σs::Invariants)